# Setup

## Install PDAL

With Conda

```
conda install -c conda-forge pdal python-pdal
```

With Docker

```
docker pull pdal/pdal:latest
```

## Move the Data on to the VM

#### Create a working directory

```
sudo mkdir -p /vol_c/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/
```

#### Move files from CyVerse DataStore onto the VM

```
iget -Kvrf /iplant/home/shared/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/ClassifiedPointCloud/
```

## Build Potree from Dockerfile

Create a Potree Docker container using the provided Dockerfile

```
docker build -t tswetnam/potreeconverter:0.1 .
```

The container is named `tswetnam/potreeconverter:0.1` with tag `0.1`

Run the container on the Classified Point Cloud `.laz` data delivered from NEON:

```
docker run \
    -v /vol_b/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/ClassifiedPointCloud/:/input \
    -v /vol_b/potree/pointclouds/:/output \
    tswetnam/potreeconverter:0.1 \
    PotreeConverter /input/ -p TEAK -o /output/TEAK_outliers
```

To view the outputs, you can move the data into the `/var/www/` folder on your VM.
Access the IP address with the `/TEAK.html` extension

Note that there are plenty of outliers in this model.

## PDAL outlier removal

We now want to filter the outliers away using `pdal` and its pipeline .json capability.

We have created a file called `outlier.json` which uses some of PDAL's filtering properties to get rid of the unwanted points.

```
{
  "pipeline":[
    "/data/{}.laz",
    {
      "type":"filters.outlier",
      "method":"statistical",
      "mean_k":12,
      "multiplier":2.2
    },
    {
      "type":"filters.range",
      "limits":"Classification![7:7]"
    },
    {
      "type":"writers.las",
      "filename":"/vol_c/filtered/{}.laz"
    }
  ]
}
```

In this example `.json` I'm using a wildcard `{}` for the file name, PDAL's [filters.outlier](https://pdal.io/stages/filters.outlier.html#filters-outlier), to remove statisical outliers, and [filters.range](ttps://pdal.io/stages/filters.outlier.html) to remove points that have been classified as class 7 (outliers).

I'm then writing the file to a new working directory.

In [ ]:
%%bash
sudo docker pull pdal/pdal:latest

To run the job over an entire collection, I'm using the Docker version of PDAL and a bash script:

```
ls *.laz | cut -d. -f1 | xargs -P6 -I{}
```

that will loop through the directory and run up to the number of cores on the VM I'm using. 

I need to create an output directory for the files at this time

```
mkdir /vol_b/filtered
```

Next, `cd` into a directory where the `*.laz` files are kept.

In [ ]:
%%bash
ls *.laz | cut -d. -f1 | xargs -P6 -I{} \
sudo docker run \
-v /vol_b/neon_data_science:/home \
-v /vol_b:/vol_b \
-v /vol_b/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/ClassifiedPointCloud/:/data \
pdal/pdal:latest pdal \
pipeline /home/pdal/outlier.json \
--readers.las.filename=/data/{}.laz \
--writers.las.filename=/vol_b/filtered/{}.laz

```
docker run \
    -v /vol_b/filtered/:/input \
    -v /vol_b/potree/pointclouds/:/output \
    tswetnam/potreeconverter:0.1 \
    PotreeConverter /input/ -p TEAK -o /output/TEAK_filtered
```

##LAStools outlier removal

Remove the Class 7s using Dockerized `las2las`
```
sudo docker run \
-v /vol_b/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/ClassifiedPointCloud/:/input \
-v /vol_b/filtered_lastools/:/output \
tswetnam/lastools:0.1 las2las \
-i /input/NEON_D17_TEAK_DP1_320000_4092000_classified_point_cloud.laz \
-o /output/NEON_D17_TEAK_DP1_320000_4092000_classified_point_cloud.laz \
-drop_class 7
```

Run Dockerized Potree on the filtered `*.laz`

```
docker run \
    -v /vol_b/filtered_lastools/:/input \
    -v /vol_b/potree/pointclouds/:/output \
    tswetnam/potreeconverter:0.1 \
    PotreeConverter /input/ -p TEAK_filtered -o /output/TEAK_filtered
```

In [ ]:
sudo docker run \
-v /vol_b/NEON_data_institute_2018/2017_Campaign/TEAK/L1/DiscreteLidar/ClassifiedPointCloud/:/input \
-v /vol_b/filtered_lastools/:/output \
tswetnam/lastools:0.1 las2las \
-i /input/* \
-o /output/* \
-drop_class 7

# Running a Potree server

Install Apache2

```
sudo apt-get update && \
sudo apt-get install apache2
```

to start the web server

```
sudo service apache2 start
```

Move the Potree directory to the `/var/www/html` directory on the VM. 

After you've moved the files, you can open a new browser tab and copy the IP address of the VM with the `<IP>/potree/pointclouds/TEAK_outliers/TEAK.html` to view the Potreee

In [5]:
%%html
<iframe src="http://129.114.19.9/potree/pointclouds/TEAK_filtered/TEAK_filtered.html" width="800" height="600"/>